# Hướng dẫn truy vấn dữ liệu thị giác dùng fiftyone

Đây là hướng dẫn dùng cho các đội tham dự AI Challenge 2023. Hướng dẫn này nhằm mục đích giới thiệu cho các đội một phương pháp cơ bản để truy vấn dữ liệu dựa trên thông tin BTC cung cấp và giới thiệu công cụ fiftyone để hỗ trợ đội thi đánh giá kết quả.

## Cài đặt ban đầu

Bạn cần cài đặt môi trường để chạy được notebook này trên máy tính cá nhân của bạn. Hướng dẫn này không bao gồm phần cài đặt môi trường. Khuyến nghị: các bạn có thể cài đặt [Anaconda](https://docs.anaconda.com/free/anaconda/install/windows/).

## Cài đặt các thư viện FiftyOne và PyTorch
Hướng dẫn này dùng fiftyone là công cụ để trực quan dữ liệu và pytorch là backend chính cho các thuật toán máy học.

### Lưu ý: Đối với các bạn dùng Windows nên dùng bản fiftyone **v0.21.4**, không nên dùng bản mới nhất!

In [1]:
! pip install fiftyone==0.21.4
! pip install torch torchvision torchaudio


     ---------------------------------------- 7.7/7.7 MB 11.9 MB/s eta 0:00:00
     ---------------------------------------- 66.9/66.9 kB ? eta 0:00:00
     ------------------------------------- 226.8/226.8 kB 13.5 MB/s eta 0:00:00
     ---------------------------------------- 78.5/78.5 kB ? eta 0:00:00
     --------------------------------------- 38.0/38.0 MB 15.2 MB/s eta 0:00:00
     ---------------------------------------- 65.9/65.9 MB 7.9 MB/s eta 0:00:00
     ------------------------------------- 468.2/468.2 kB 30.5 MB/s eta 0:00:00
     ---------------------------------------- 41.5/41.5 kB ? eta 0:00:00
     ---------------------------------------- 108.9/108.9 kB ? eta 0:00:00
     ---------------------------------------- 53.1/53.1 kB ? eta 0:00:00
     ---------------------------------------- 58.2/58.2 kB 3.0 MB/s eta 0:00:00
     ------------------------------------- 192.5/192.5 kB 11.4 MB/s eta 0:00:00
     -------------------------------------- 135.8/135.8 kB 7.8 MB/s eta 0:

Load dữ liệu keyframe từ thư mục chứa keyframe. Mỗi ảnh và thông tin đi kèm sau này sẽ được lưu trữ trong một Sample. Tất cả các Sample được lưu trong Dataset.

In [1]:
import fiftyone as fo
import fiftyone.brain as fob
import numpy as np
from glob import glob
import json
import os

Load dữ liệu keyframe từ thư mục chứa keyframe. Trong hướng dẫn này tất cả các file Keyframes_L*.zip được giải nén vào thư mục `D:\AIC\Keyframes`. Mỗi ảnh và thông tin đi kèm sau này sẽ được lưu trữ trong một `Sample`. Tất cả các `Sample` được lưu trong `Dataset`.

In [2]:
dataset = fo.Dataset.from_images_dir('E:\\BKHIGHTECH\\Challenge\\AIC23 HCM\\Dataset\\Test\\keyframes', name=None, tags=None, recursive=True)

 100% |█████████████| 86940/86940 [17.6s elapsed, 0s remaining, 5.1K samples/s]      


Sau khi dữ liệu đã load lên xong. Bạn có thể truy cập vào đường vào ứng dụng web của fiftyone từ [http://localhost:5151](http://localhost:5151)

In [4]:
session = fo.launch_app(dataset, auto=False)


Could not connect session, trying again in 10 seconds

Session launched. Run `session.show()` to open the App in a cell output.


Hoặc bạn có thể chạy cell bên dưới để mở tab mới cho ứng dụng web fiftyone

In [5]:
session.open_tab()

<IPython.core.display.Javascript object>

### Trích xuất thêm thông tin tên của video và frameid
Thông tin `video` và `frameid` sẽ được lấy từ tên của tập tin keyframe.

In [6]:
for sample in dataset:
    _, sample['video'], sample['frameid'] = sample['filepath'][:-4].rsplit('\\', 2)
    sample.save()

Bạn có thể xem `Sample` đầu tiên của `Dataset` bằng lệnh sau:

In [7]:
print(dataset.first())

<Sample: {
    'id': '64fec309f3e66a3716a46067',
    'media_type': 'image',
    'filepath': 'E:\\BKHIGHTECH\\Challenge\\AIC23 HCM\\Dataset\\Test\\keyframes\\L01_V001\\0001.jpg',
    'tags': [],
    'metadata': None,
    'video': 'L01_V001',
    'frameid': '0001',
}>


### Thêm thông tin kết quả của object detection.

Bước này có thể tốn của bạn nhiều thời gian để đọc hết tất cả các dữ liệu về object detection. Bạn có thể bỏ qua cell này và chạy cell này sau nếu muốn thử thêm các thông tin về vector CLIP embedding trước.

In [ ]:
for sample in dataset:
    object_path = f"E:\\BKHIGHTECH\\Challenge\\AIC23 HCM\\Dataset\\Test\\objects\\{sample['video']}\\{sample['frameid']}.json"
    with open(object_path) as jsonfile:
        det_data = json.load(jsonfile)
    detections = []
    for cls, box, score in zip(det_data['detection_class_entities'], det_data['detection_boxes'], det_data['detection_scores']):
        # Convert to [top-left-x, top-left-y, width, height]
        boxf = [float(box[1]), float(box[0]), float(box[3]) - float(box[1]), float(box[2]) - float(box[0])]
        scoref = float(score)

        # Only add objects with confidence > 0.4
        if scoref > 0.4:
            detections.append(
                fo.Detection(
                    label=cls,
                    bounding_box= boxf,
                    confidence=float(score)
                )
            )
    sample["object_faster_rcnn"] = fo.Detections(detections=detections)
    sample.save()


### Thêm thông tin CLIP embedding.

In [ ]:
all_keyframe = glob('E:\\BKHIGHTECH\\Challenge\\AIC23 HCM\\Dataset\\Test\\keyframes\\*\\*.jpg')
video_keyframe_dict = {}
all_video = glob('E:\\BKHIGHTECH\\Challenge\\AIC23 HCM\\Dataset\\Test\\keyframes\\*')
all_video = [v.rsplit('\\',1)[-1] for v in all_video]

Đọc thông tin clip embedding được cung cấp.

Lưu ý: Các bạn cần tải đúng bản CLIP embedding từ model **CLIP ViT-B/32**

Tạo dictionary `video_keyframe_dict` với `video_keyframe_dict[video]` thông tin danh sách `keyframe` của `video`

In [ ]:
for kf in all_keyframe:
    _, vid, kf = kf[:-4].rsplit('\\',2)
    if vid not in video_keyframe_dict.keys():
        video_keyframe_dict[vid] = [kf]
    else:
        video_keyframe_dict[vid].append(kf)

Do thông tin vector CLIP embedding được cung cấp được lưu theo từng video nhầm mục đích tối ưu thời gian đọc dữ liệu. Cần sort lại danh sách `keyframe` của từng `video` để đảm bảo thứ tự đọc đúng với vector embedding được cung cấp.

In [ ]:
for k,v in video_keyframe_dict.items():
    video_keyframe_dict[k] = sorted(v)

Tạo dictionary `embedding_dict` với `embedding_dict[video][keyframe]` lưu thông tin vector CLIP embedding của `keyframe` trong `video` tương ứng

In [ ]:
embedding_dict = {}
for v in all_video:
    clip_path = f'E:\\BKHIGHTECH\\Challenge\\AIC23 HCM\\Dataset\\Test\\clip-features-vit-b32\\{v}.npy'
    a = np.load(clip_path)
    embedding_dict[v] = {}
    for i,k in enumerate(video_keyframe_dict[v]):
        embedding_dict[v][k] = a[i]


Tạo danh sách `clip_embedding` ứng với danh sách `sample` trong `dataset`.

In [ ]:
clip_embeddings = []
for sample in dataset:
    clip_embedding = embedding_dict[sample['video']][sample['frameid']]
    clip_embeddings.append(clip_embedding)


In [ ]:
fob.compute_similarity(
    dataset,
    model="clip-vit-base32-torch",      # store model's name for future use
    embeddings=clip_embeddings,          # precomputed image embeddings
    brain_key="img_sim",
)

## Từ đây các bạn có thể thử các tính năng search, filter trên ứng dụng fiftyone.

In [ ]:
# Bạn cần phải cài version umap-learn hỗ trợ.
fob.compute_visualization(
    dataset,
    embeddings=clip_embeddings,
    brain_key="img_viz"
)
